Copyright 2024 Google, LLC. This software is provided as-is,
without warranty or representation for any use or purpose. Your
use of it is subject to your agreement with Google.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# How to use Batch Predicitons with Gemini

This notebook outlines how to interact with Vertex AI's Gemini models to call external API's using Function Calling. More info can be found at https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/batch-prediction-gemini

## Prepare the python development environment

First, let's identify any project specific variables to customize this notebook to your GCP environment. Change YOUR_PROJECT_ID with your own GCP project ID.

In [ ]:
project_id = "YOUR_PROJECT_ID"
location = "global"
region = "us-central1"
bq_dataset_id = "gemini_batch_test"
bq_input_table = "batch_input_table"
bq_output_table = "gem_batch_output"
batch_job_name = "gem_batch_job_001"
model_ver = "gemini-1.0-pro-002"

Install any needed python modules from our requirements.txt file. Most Vertex Workbench environments include all the packages we'll be using, but if you are using an external Jupyter Notebook or require any additional packages for your own needs, you can simply add them to the included requirements.txt file an run the folloiwng commands.

In [ ]:
#pip install -r requirements.txt

Now we will import all required modules. For our purpose, we will be utilizing the following:

- requests - This module will allow us to interact directly with external REST API's. 


In [ ]:
import time
import google.auth.transport.requests
import google.auth
import base64
import requests
import json


## Define the Gemini Batch Prediciton Request

Next we will define the request header and data to be sent to the Gemini Batch Prediction API

In [ ]:
credentials, project_id = google.auth.default()
auth_req = google.auth.transport.requests.Request()
credentials.refresh(auth_req)

In [ ]:
access_token = credentials.token

In [ ]:
headers = {
        'Authorization': 'Bearer ' + access_token,
        'Content-Type': 'application/json; charset=utf-8'
    }

In [ ]:
url = f'https://{region}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{region}/batchPredictionJobs'

In [ ]:
request_body = f'''{{
    "name": "{batch_job_name}",
    "displayName": "{batch_job_name}",
    "model": "publishers/google/models/{model_ver}",
    "inputConfig": {{
      "instancesFormat":"bigquery",
      "bigquerySource":{{
        "inputUri" : "bq://{project_id}.{bq_dataset_id}.{bq_input_table}"
      }}
    }},
    "outputConfig": {{
      "predictionsFormat":"bigquery",
      "bigqueryDestination":{{
        "outputUri": "bq://{project_id}.{bq_dataset_id}.{bq_output_table}"
        }}
    }}
}}'''

Submit the request to the RestAPI

In [ ]:
r = requests.post(url, data=request_body, headers=headers)

Print the status code and details of the prediction job. You can view the status in the GCP Cloud Console under Vertex AI -> Batch Predictions and select the appropraite region. 

In [ ]:
print(r.status_code)
print(r.content)